In [13]:
# Import libraries
import cfbd
from cfbd.rest import ApiException
from pprint import pprint
import os
from dotenv import load_dotenv

# load environment variables
load_dotenv()

# get API key
API_KEY = os.getenv("API_KEY")
BASE_URL = "https://api.collegefootballdata.com"

# Remove 'Bearer ' prefix if it exists
if API_KEY.startswith("Bearer "):
    API_KEY = API_KEY.split("Bearer ")[1]

# Print first few characters of API key to verify it's loaded
print(f"API Key (first 5 chars): {API_KEY[:5]}...")

# Configure API key authorization
configuration = cfbd.Configuration()
configuration.api_key['Authorization'] = API_KEY
configuration.api_key_prefix['Authorization'] = 'Bearer'


API Key (first 5 chars): 5MLmI...


In [14]:
# Create an instance of the API class
api_instance = cfbd.GamesApi(cfbd.ApiClient(configuration))

# Print the headers that will be sent (excluding the actual API key)
headers = api_instance.api_client.default_headers
safe_headers = {k: v if k != 'Authorization' else 'Bearer [REDACTED]' for k, v in headers.items()}
print("Headers being sent:")
pprint(safe_headers)

try:
    # Fetch games for 2023
    games = api_instance.get_games(year=2023)
    pprint(games[:5])  # Print first 5 games
except ApiException as e:
    print(f"Exception when calling GamesApi->get_games: {e}\n")
    print(f"Response body: {e.body}\n")

Headers being sent:
{'User-Agent': 'Swagger-Codegen/4.5.2/python'}
[{'attendance': None,
 'away_conference': None,
 'away_division': None,
 'away_id': 125762,
 'away_line_scores': None,
 'away_points': None,
 'away_post_win_prob': None,
 'away_postgame_elo': None,
 'away_pregame_elo': None,
 'away_team': 'Florida Memorial University',
 'completed': True,
 'conference_game': False,
 'excitement_index': None,
 'highlights': None,
 'home_conference': 'USA South',
 'home_division': 'iii',
 'home_id': 548,
 'home_line_scores': None,
 'home_points': None,
 'home_post_win_prob': None,
 'home_postgame_elo': None,
 'home_pregame_elo': None,
 'home_team': 'LaGrange College',
 'id': 401550883,
 'neutral_site': False,
 'notes': None,
 'season': 2023,
 'season_type': 'regular',
 'start_date': '2023-08-26T17:00:00.000Z',
 'start_time_tbd': False,
 'venue': 'Callaway Stadium',
 'venue_id': 5910,
 'week': 1},
 {'attendance': 49000,
 'away_conference': 'American Athletic',
 'away_division': 'fbs',
 'aw